<a href="https://colab.research.google.com/github/MSechineli/BCC36B---Compiladores/blob/main/tppcompiler/01-compiladores-analise-lexica-tpplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Léxica

## Preparação do Ambiente

*   Instalação do [PLY](https://www.dabeaz.com/ply/ply.html)


In [1]:
!pip install ply

     |████████████████████████████████| 49 kB 3.8 MB/s 


In [2]:
!jupyter nbextension install https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip  --user
!jupyter nbextension enable highlighter/highlighter

Downloading: https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip -> /tmp/tmpqu0qmnca/highlighter.zip
Extracting: /tmp/tmpqu0qmnca/highlighter.zip -> /root/.local/share/jupyter/nbextensions
Enabling notebook extension highlighter/highlighter...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [3]:
%%javascript
require("base/js/utils").load_extensions("highlighter/highlighter")

<IPython.core.display.Javascript object>

In [4]:
from sys import argv, exit

import logging
logging.basicConfig(
     level = logging.DEBUG,
     filename = "log.txt",
     filemode = "w",
     format = "%(filename)10s:%(lineno)4d:%(message)s"
)
log = logging.getLogger()

import ply.lex as lex
from ply.lex import TOKEN

In [5]:
tokens = [
    "ID",  # identificador
    # numerais
    "NUM_NOTACAO_CIENTIFICA",  # ponto flutuante em notaçao científica
    "NUM_PONTO_FLUTUANTE",  # ponto flutuate
    "NUM_INTEIRO",  # inteiro
    # operadores binarios
    "MAIS",  # +
    "MENOS",  # -
    "MULTIPLICACAO",  # *
    "DIVISAO",  # /
    "E_LOGICO",  # &&
    "OU_LOGICO",  # ||
    "DIFERENCA",  # <>
    "MENOR_IGUAL",  # <=
    "MAIOR_IGUAL",  # >=
    "MENOR",  # <
    "MAIOR",  # >
    "IGUAL",  # =
    # operadores unarios
    "NEGACAO",  # !
    # simbolos
    "ABRE_PARENTESE",  # (
    "FECHA_PARENTESE",  # )
    "ABRE_COLCHETE",  # [
    "FECHA_COLCHETE",  # ]
    "VIRGULA",  # ,
    "DOIS_PONTOS",  # :
    "ATRIBUICAO",  # :=
    # 'COMENTARIO', # {***}
]

In [6]:
reserved_words = {
    "se": "SE",
    "então": "ENTAO",
    "senão": "SENAO",
    "fim": "FIM",
    "repita": "REPITA",
    "flutuante": "FLUTUANTE",
    "retorna": "RETORNA",
    "até": "ATE",
    "leia": "LEIA",
    "escreva": "ESCREVA",
    "inteiro": "INTEIRO",
}

tokens = tokens + list(reserved_words.values())

In [7]:
digito = r"([0-9])"
letra = r"([a-zA-ZáÁãÃàÀéÉíÍóÓõÕ])"
sinal = r"([\-\+]?)"

""" 
    id deve começar com uma letra
"""
id = (
    r"(" + letra + r"(" + digito + r"+|_|" + letra + r")*)"
)  # o mesmo que '((letra)(letra|_|([0-9]))*)'

# inteiro = r"(" + sinal + digito + r"+)"
# inteiro = r"(" + digito + r"+)"
inteiro = r"\d+"

flutuante = (
    # r"(" + digito + r"+\." + digito + r"+?)"
    # (([-\+]?)([0-9]+)\.([0-9]+))'
    r'\d+[eE][-+]?\d+|(\.\d+|\d+\.\d*)([eE][-+]?\d+)?'
    # r'[-+]?[0-9]+(\.([0-9]+)?)'
    #r'[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?'
    #r"(([-\+]?)([0-9]+)\.([0-9]+))"
    )

notacao_cientifica = (
    r"(" + sinal + r"([1-9])\." + digito + r"+[eE]" + sinal + digito + r"+)"
)  # o mesmo que '(([-\+]?)([1-9])\.([0-9])+[eE]([-\+]?)([0-9]+))'

In [8]:
 
# Expressões Regulaes para tokens simples.
# Símbolos.
t_MAIS = r'\+'
t_MENOS = r'-'
t_MULTIPLICACAO = r'\*'
t_DIVISAO = r'/'
t_ABRE_PARENTESE = r'\('
t_FECHA_PARENTESE = r'\)'
t_ABRE_COLCHETE = r'\['
t_FECHA_COLCHETE = r'\]'
t_VIRGULA = r','
t_ATRIBUICAO = r':='
t_DOIS_PONTOS = r':'

# Operadores Lógicos.
t_E_LOGICO = r'&&'
t_OU_LOGICO = r'\|\|'
t_NEGACAO = r'!'

# Operadores Relacionais.
t_DIFERENCA = r'<>'
t_MENOR_IGUAL = r'<='
t_MAIOR_IGUAL = r'>='
t_MENOR = r'<'
t_MAIOR = r'>'
t_IGUAL = r'='

In [9]:
@TOKEN(id)
def t_ID(token):
    token.type = reserved_words.get(
        token.value, "ID"
    )  # não é necessário fazer regras/regex para cada palavra reservada
    # se o token não for uma palavra reservada automaticamente é um id
    # As palavras reservadas têm precedências sobre os ids

    return token

@TOKEN(notacao_cientifica)
def t_NUM_NOTACAO_CIENTIFICA(token):
    return token

@TOKEN(flutuante)
def t_NUM_PONTO_FLUTUANTE(token):
    return token

@TOKEN(inteiro)
def t_NUM_INTEIRO(token):
    return token

In [10]:
t_ignore = " \t"

# t_COMENTARIO = r'(\{((.|\n)*?)\})'
# para poder contar as quebras de linha dentro dos comentarios
def t_COMENTARIO(token):
    r"(\{((.|\n)*?)\})"
    token.lexer.lineno += token.value.count("\n")
    # return token

def t_newline(token):
    r"\n+"
    token.lexer.lineno += len(token.value)

def define_column(input, lexpos):
    begin_line = input.rfind("\n", 0, lexpos) + 1
    return (lexpos - begin_line) + 1

In [11]:
def t_error(token):

    # file = token.lexer.filename
    line = token.lineno
    # column = define_column(token.lexer.backup_data, token.lexpos)
    message = "Caracter ilegal '%s'" % token.value[0]

    # print(f"[{file}]:[{line},{column}]: {message}.") 
    print(message)

    token.lexer.skip(1)

    # token.lexer.has_error = Trueb

In [12]:
def main():
    # argv[1] = 'teste.tpp'
    aux = argv[1].split('.')
    if aux[-1] != 'tpp':
      raise IOError("Not a .tpp file!")
    data = open(argv[1])

    source_file = data.read()
    lexer.input(source_file)

    # Tokenize
    while True:
      tok = lexer.token()
      if not tok: 
        break      # No more input
      print(tok)
      # print(tok.type)
      #print(tok.value)

In [13]:
# Build the lexer.
__file__ = "01-compiladores-analise-lexica-tpplex.ipynb"
lexer = lex.lex(optimize=True,debug=True,debuglog=log)

if __name__ == "__main__":
    main()

OSError: ignored

In [ ]:
%%writefile teste.tpp

inteiro: a[10]
flutuante: b

inteiro func1(inteiro:x, flutuante:y)
  inteiro: res
  se (x > y) então
    res := x + y
  senão
    res := x * y
  fim
  retorna(res)
fim

func2(inteiro:z, flutuante:w)
  a := z
  b := w
fim

inteiro principal()
  inteiro: x,y
  flutuante: w
  a := 10 + 2
  leia(x)
  leia(w)
  w := .6 + 1.
  func2(1, 2.5)
  b := func1(x,w)
  escreva(b)
  retorna(0)
fim


Writing teste.tpp


In [ ]:
! wget https://raw.githubusercontent.com/rogerioag/tutorial-de-compiladores/master/tppcompiler/tpplex.py

--2021-03-10 20:32:18--  https://raw.githubusercontent.com/rogerioag/tutorial-de-compiladores/master/tppcompiler/tpplex.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4628 (4.5K) [text/plain]
Saving to: ‘tpplex.py’

tpplex.py           100%[===================>]   4.52K  --.-KB/s    in 0s      

2021-03-10 20:32:18 (55.6 MB/s) - ‘tpplex.py’ saved [4628/4628]



In [ ]:
!python tpplex.py teste.tpp

INTEIRO
DOIS_PONTOS
ID
ABRE_COLCHETE
NUM_INTEIRO
FECHA_COLCHETE
FLUTUANTE
DOIS_PONTOS
ID
INTEIRO
ID
ABRE_PARENTESE
INTEIRO
DOIS_PONTOS
ID
VIRGULA
FLUTUANTE
DOIS_PONTOS
ID
FECHA_PARENTESE
INTEIRO
DOIS_PONTOS
ID
SE
ABRE_PARENTESE
ID
MAIOR
ID
FECHA_PARENTESE
ENTAO
ID
ATRIBUICAO
ID
MAIS
ID
SENAO
ID
ATRIBUICAO
ID
MULTIPLICACAO
ID
FIM
RETORNA
ABRE_PARENTESE
ID
FECHA_PARENTESE
FIM
ID
ABRE_PARENTESE
INTEIRO
DOIS_PONTOS
ID
VIRGULA
FLUTUANTE
DOIS_PONTOS
ID
FECHA_PARENTESE
ID
ATRIBUICAO
ID
ID
ATRIBUICAO
ID
FIM
INTEIRO
ID
ABRE_PARENTESE
FECHA_PARENTESE
INTEIRO
DOIS_PONTOS
ID
VIRGULA
ID
FLUTUANTE
DOIS_PONTOS
ID
ID
ATRIBUICAO
NUM_INTEIRO
MAIS
NUM_INTEIRO
LEIA
ABRE_PARENTESE
ID
FECHA_PARENTESE
LEIA
ABRE_PARENTESE
ID
FECHA_PARENTESE
ID
ATRIBUICAO
NUM_PONTO_FLUTUANTE
MAIS
NUM_PONTO_FLUTUANTE
ID
ABRE_PARENTESE
NUM_INTEIRO
VIRGULA
NUM_PONTO_FLUTUANTE
FECHA_PARENTESE
ID
ATRIBUICAO
ID
ABRE_PARENTESE
ID
VIRGULA
ID
FECHA_PARENTESE
ESCREVA
ABRE_PARENTESE
ID
FECHA_PARENTESE
RETORNA
ABRE_PARENTESE
NUM_INTEIRO
F

In [ ]:
%%writefile teste-003.tpp
inteiro principal()
  a :=+1
  a := a + b
  b:= 3 + a
  c:= +3
fim

Overwriting teste-003.tpp


In [ ]:
!python tpplex.py teste-003.tpp

INTEIRO
ID
ABRE_PARENTESE
FECHA_PARENTESE
ID
ATRIBUICAO
MAIS
NUM_INTEIRO
ID
ATRIBUICAO
ID
MAIS
ID
ID
ATRIBUICAO
NUM_INTEIRO
MAIS
ID
ID
ATRIBUICAO
MAIS
NUM_INTEIRO
FIM
